In [3]:
import json
from os.path import join
import pickle
import codecs
import pandas as pd
import re
import os
import time
from datetime import datetime
start_time = datetime.now()

In [16]:
# def precessname(name):   
# #    name = name.strip().lower().replace(' ', '_')
#     name = str(name).strip().lower()
#     name = re.sub('[!“”"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~—～’]+',' ', name)
#     name = re.sub( r'\s{2,}', ' ', name).strip()
#     name = name.replace(' ', '_')
#     return name
# precessname('liu*]][[]]*zhi.ping.')
# # print(''.join('liu_zhiping'.split('_')))

'liu_zhi_ping'

In [4]:
################# Load and Save Data ################

def load_json(rfdir, rfname):
    with codecs.open(join(rfdir, rfname), 'r', encoding='utf-8') as rf:
        return json.load(rf)


def dump_json(obj, wfpath, wfname, indent=None):
    with codecs.open(join(wfpath, wfname), 'w', encoding='utf-8') as wf:
        json.dump(obj, wf, ensure_ascii=False, indent=indent)

def precess_df(data):
    data = data.dropna(subset=['name']).copy()
    data.drop_duplicates(subset=['name','uid'],keep='first',inplace=True)
    return data

def precessname(name):   
#    name = name.strip().lower().replace(' ', '_')
    name = str(name).strip().lower()
    name = re.sub('[!“”"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~—～’]+',' ', name)
    name = re.sub( r'\s{2,}', ' ', name).strip()
    name = name.replace(' ', '')
    return name

def reverse_name(name):
    name = precessname(name)
    name = name.split('_')
    reverse_name = '_'.join(name[::-1])
    return reverse_name

def normname(name):
    name = ''.join(name.split('_'))
    return name

def get_pinyin(name):
    name = precessname(pinyin.get(name, format='strip', delimiter="_")).replace('_','')
    return name

##利用pandas的groupby函数来获取需要的数据集
def stand_groupby(dataset):
    disamb ={}
    for name,group in dataset.groupby(['name']):
        if normname(name)  in disamb:
            disamb[normname(name)] += list(zip(*[group[c].values.tolist() for c in ['uid', 'org_name']]))
        elif normname(reverse_name(name)) not in disamb:
            disamb[normname(name)]  = list(zip(*[group[c].values.tolist() for c in ['uid', 'org_name']]))
        else:        
            disamb[normname(reverse_name(name))]+=list(zip(*[group[c].values.tolist() for c in ['uid', 'org_name']]))
    return disamb

##利用pandas的groupby函数来获取需要的数据集
def stand_groupby_norm(dataset):
    disamb ={}
    for name,group in dataset.groupby(['name']):
        if name not in disamb:
            disamb[name] = list(zip(*[group[c].values.tolist() for c in ['uid', 'org_name']]))
        else:
            disamb[name] += list(zip(*[group[c].values.tolist() for c in ['uid', 'org_name']]))
    return disamb



In [6]:
def dump_data_to_namepub():
    yy = pd.read_csv('data/t_018_uid_name_org_preccess1_10.csv',encoding='UTF-8')
    print('利用pandas读取数据为csv文件花费时间: ',datetime.now()-start_time)
    
    yy['name'] = yy['name'].map(precessname)
    print(yy.columns)
#     yy['norm_name'] = yy['name'].map(normname)
    print('本轮消歧涉及到的论文数量：%d'%(len(yy)))
    name_pubs_1_10 = stand_groupby_norm(yy)
#     print(name_pubs_50['aaronsonnk'])
    del yy
    print('本轮消歧之前的作者数量： %d'%(len(name_pubs_1_10)))
    dump_json(name_pubs_1_10,'data', 'sci_author_data_1_10_all.json')
    print('csv转化为同名论文集格式总耗时 : ',datetime.now()-start_time) 
dump_data_to_namepub()

利用pandas读取数据为csv文件花费时间:  0:01:07.649249
Index(['uid', 'name', 'org_name'], dtype='object')
本轮消歧涉及到的论文数量：27034623
本轮消歧之前的作者数量： 11330581
csv转化为同名论文集格式总耗时 :  1:01:00.689944
